<a href="https://colab.research.google.com/github/cemyyldz/pdf-copy-project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Ortam Kurulumu ve Kütüphane Yükleme
Bu bölümde, projenin temel ve destekleyici tüm kütüphaneleri kurulur. Birebir (Senile) replikasyon için, görsel çıkarımı (pdfplumber) ve görsel manipülasyonu (Pillow) sağlayan destekleyici kütüphaneler eklenmiştir.

In [ ]:

import os
print("Gerekli tüm kütüphaneler kuruluyor...")
!pip install PyPDF2 pdfminer.six reportlab pdfplumber Pillow
print("Kurulum tamamlandı.")


Gerekli tüm kütüphaneler kuruluyor...
Kurulum tamamlandı.


2. Aşama: Yapısal Veri Çıkarımı (Metin, Şekil ve Görsel)Bu aşamada, PDF'ten metin koordinatları (pdfminer.six) ve görseller (pdfplumber) dahil olmak üzere tüm yapısal veriler tek bir project_data yapısına çıkarılır. Görsel konumlandırma sorunlarını çözmek için y ekseni düzeltmeleri bu kısımda yapılır.

In [ ]:

import os
from PyPDF2 import PdfReader
from PIL import Image
import io
import re
import shutil

from pdfminer.layout import LTTextContainer, LTTextLine, LTChar, LTLine, LTRect
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from reportlab.pdfgen import canvas
from reportlab.lib.colors import black
import pdfplumber

PDF_INPUT_FILE = "file.pdf"
PDF_OUTPUT_FILE = "output_birebir_FINAL_V6.pdf" # Yeni dosya adı

project_data = {'text_blocks': [], 'lines': [], 'rects': [], 'images': []}
page_w, page_h = 0, 0

FONT_MAPPING = {
    'REGULAR': 'Helvetica', 'PRO-REGULAR': 'Helvetica',
    'BOLD': 'Helvetica-Bold', 'PRO-BOLD': 'Helvetica-Bold',
    'ITALIC': 'Helvetica-Oblique', 'PRO-ITALIC': 'Helvetica-Oblique',
    'ARIALMT': 'Helvetica', 'TIMESNEWROMAN': 'Times-Roman', 'COURIER': 'Courier',
}


def get_page_dimensions(file_path):
    global page_w, page_h
    try:
        reader = PdfReader(file_path)
        page = reader.pages[0]
        media_box = page.mediabox
        page_w, page_h = float(media_box.width), float(media_box.height)
        return True
    except Exception as e:
        return False

def extract_all_structural_elements(file_path):
    rsrcmgr = PDFResourceManager()
    laparams = LAParams(line_overlap=0.5, char_margin=2.0, word_margin=0.1, line_margin=0.5, boxes_flow=0.5, detect_vertical=False)
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    try:
        with open(file_path, 'rb') as fp:
            for page in PDFPage.get_pages(fp):
                interpreter.process_page(page)
                layout = device.get_result()
                for element in layout:
                    if isinstance(element, LTTextContainer):
                        for text_line in element:
                            if isinstance(text_line, LTTextLine):
                                current_text = ""
                                last_char_data = None
                                for character in text_line:
                                    if isinstance(character, LTChar):
                                        current_text += character.get_text()
                                        if last_char_data is None:
                                            last_char_data = {'x': character.bbox[0], 'y': character.bbox[1], 'font': character.fontname, 'size': character.size}
                                if current_text and last_char_data:
                                    data = last_char_data
                                    data['text'] = current_text
                                    project_data['text_blocks'].append(data)
                    elif isinstance(element, LTLine):
                        project_data['lines'].append({'x0': element.bbox[0], 'y0': element.bbox[1], 'x1': element.bbox[2], 'y1': element.bbox[3], 'width': element.linewidth })
                    elif isinstance(element, LTRect):
                        project_data['rects'].append({'x0': element.bbox[0], 'y0': element.bbox[1], 'x1': element.bbox[2], 'y1': element.bbox[3], 'width': element.linewidth })
    except Exception as e:
        return

def extract_images_with_pdfplumber(file_path):
    """Görselleri çıkarır, dikey çevirir ve ReportLab Y koordinatını hesaplar."""

    temp_dir = 'temp_images'
    os.makedirs(temp_dir, exist_ok=True)

    try:
        with pdfplumber.open(file_path) as pdf:
            page = pdf.pages[0]

            for img_data in page.images:
                x0, y0, x1, y1 = img_data['x0'], img_data['y0'], img_data['x1'], img_data['y1']

                if 'stream' in img_data:
                    img_bytes = img_data['stream'].get_data()
                    temp_filename = os.path.join(temp_dir, f"img_{len(project_data['images'])}.png")

                    image = Image.open(io.BytesIO(img_bytes))


                    image = image.transpose(Image.FLIP_TOP_BOTTOM)

                    image.save(temp_filename)

                    y_reportlab = y0

                    project_data['images'].append({
                        'x': x0,
                        'y': y_reportlab,
                        'width': x1 - x0,
                        'height': y1 - y0,
                        'file': temp_filename
                    })

    except Exception as e:
        print(f"pdfplumber ile görsel çıkarılırken hata: {e}")


def create_output_pdf_birebir_final(output_file):

    c = canvas.Canvas(output_file, pagesize=(page_w, page_h))
    c.setStrokeColor(black)
    c.setFillColor(black)

    MIN_LINE_LENGTH = 10
    for line in project_data.get('lines', []):
        length = max(abs(line['x1'] - line['x0']), abs(line['y1'] - line['y0']))
        if length >= MIN_LINE_LENGTH:
            c.setLineWidth(line['width'] if line['width'] > 0 else 0.5)
            c.line(line['x0'], line['y0'], line['x1'], line['y1'])

    for rect in project_data.get('rects', []):
        x, y, x1, y1 = rect['x0'], rect['y0'], rect['x1'], rect['y1']
        width = x1 - x
        height = y1 - y
        if width > 5 or height > 5:
            c.setLineWidth(rect['width'] if rect['width'] > 0 else 0.5)
            c.rect(x, y, width, height, stroke=1, fill=0)

    for img in project_data.get('images', []):

        try:
            c.drawImage(
                img['file'],
                img['x'],
                img['y'],
                width=img['width'],
                height=img['height'],
                mask='auto'
            )
        except Exception as e:
            print(f"Görsel çiziminde hata (X: {img['x']:.2f}, Y: {img['y']:.2f}): {e}")

    for block in project_data.get('text_blocks', []):
        x, y, font_name, font_size, text = block['x'], block['y'], block['font'], block['size'], block['text']

        reportlab_font = 'Helvetica'
        normalized_name = font_name.upper()
        for key, value in FONT_MAPPING.items():
            if key in normalized_name:
                reportlab_font = value
                break

        c.setFillColor(black)
        c.setFont(reportlab_font, font_size)
        c.drawString(x, y, text)

    c.save()
    shutil.rmtree('temp_images', ignore_errors=True)
    return output_file

print("--- NİHAİ UYGULAMA BAŞLIYOR ---")
if get_page_dimensions(PDF_INPUT_FILE):
    extract_all_structural_elements(PDF_INPUT_FILE)
    extract_images_with_pdfplumber(PDF_INPUT_FILE)

    if page_w != 0 and page_h != 0:
        print(f"Sayfa Boyutları: {page_w:.2f} x {page_h:.2f}")
        print(f"Görsel Sayısı: {len(project_data['images'])}")

        if project_data['images']:
             print(f"İlk Görselin NİHAİ ReportLab Y Konumu: {project_data['images'][0]['y']:.2f}")

        print("\n--- AŞAMA 3: NİHAİ SENILE ÜRETİM BAŞLIYOR ---")
        try:
            output_file_name = create_output_pdf_birebir_final(PDF_OUTPUT_FILE)

            print(f"\n✅ NİHAİ BAŞARI: Senile replikasyon '{output_file_name}' ile tamamlandı.")
            print("Görsel konum (Y ekseni) ReportLab'e göre yeniden düzeltilmiştir.")

            from google.colab import files
            files.download(output_file_name)

        except Exception as e:
            print(f"ReportLab ile PDF oluşturulurken kritik bir hata oluştu: {e}")
    else:
        print("HATA: Boyutlar sıfır döndü. İşleme devam edilemiyor.")
else:
    print("HATA: Boyutlar çıkarılamadığı için işleme devam edilemiyor.")

--- NİHAİ UYGULAMA BAŞLIYOR ---
Sayfa Boyutları: 595.50 x 842.25
Görsel Sayısı: 2
İlk Görselin NİHAİ ReportLab Y Konumu: 593.26

--- AŞAMA 3: NİHAİ SENILE ÜRETİM BAŞLIYOR ---
PDF dosyası 'output_birebir_FINAL_V6.pdf' başarıyla oluşturuldu.

✅ NİHAİ BAŞARI: Senile replikasyon 'output_birebir_FINAL_V6.pdf' ile tamamlandı.
Görsel konum (Y ekseni) ReportLab'e göre yeniden düzeltilmiştir.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>